<a href="https://colab.research.google.com/github/inuinana/pytorch_practice/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Akihiro Inui 2019 June
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
        self.optimizer = optim.SGD(self.parameters(), lr=0.01, momentum=0.5)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


    def train(self, epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            self.optimizer.zero_grad()
            output = self(data)
            self.loss = F.nll_loss(output, target)
            self.loss.backward()
            self.optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), self.loss.data))


    def test(self):
        self.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            output = self(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).data
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

# main
if __name__ == "__main__":
    # our model
    model = CNN()
    
    # Train and test model
    for epoch in range(1, 10):
        model.train(epoch)
        model.test()


0it [00:00, ?it/s]

9920512it [00:01, 8727321.44it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134576.51it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51133.38it/s]            
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314281
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300726
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291029
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.287144
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.281835
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.274881
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.257197
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.232050
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.229403
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.220390
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.184065
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.145991
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.114761
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.021271
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.830314
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.625047
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.430758
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.351720
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.019772
Train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1182, Accuracy: 9650/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.127514
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.091300
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.156144
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.199324
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.079518
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.146436
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.165773
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.049504
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.123354
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.141640
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.086249
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.148986
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.064612
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.363613
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.026282
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.137389
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.057133
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.072775
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.249552
Train Epoch: 2 [12